# DeutscheWelle Data Engineering Challenge

9.05.2023


# Task 1

## Connect via Gmail API



In [ ]:
## Script that dowloads emails from gmail inbox

## from https://developers.google.com/gmail/api/reference/rest/v1/users.messages.attachments/get 
# # Method to download email attachments from gmail api
##
# userId dwdechallenge@gmail.com / me (authenticated user)
# messageId 
# id  2022-09-15-23-04-08-EDT-Historical-Report-ENTRFacebook-2022-06-16--2022-09-16.csv 
#GET https://gmail.googleapis.com/gmail/v1/users/{userId}/messages/{messageId}/attachments/{id}

In [8]:
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from googleapiclient.errors import HttpError
import base64
import os
import time

In [9]:
#works! 
## 1) find email for the certain query i.e. 'reports' in emails subject/body
## 2) get message ids
## 3) get attachments ids
## 4) downloads attachments for every found email

def search_email(service, query_string,label_ids=[]):
    #try:
        message_list_response = service.users().messages().list(
            userId='me',
            labelIds=label_ids,
            q=query_string
        ).execute()

        message_items=message_list_response.get('messages')
        nextPageToken=message_list_response.get('nextPageToken')

        while nextPageToken:
            message_list_response = service.users().messages().list(
                    userId='me',
                    labelIds=label_ids,
                    q=query_string,
                    pageToken=nextPageToken
            ).execute()    

            message_items.extend(message_list_response.get('messages'))
            nextPageToken=message_items.get('nextPageToken')
        return message_items
    #except Exception as e:
    #    return 'No emails found'

def get_message_detail(msg_id,msg_format='metadata',metadata_headers: list=None):
    message_detail=service.users().messages().get(
        userId='me',
        id=msg_id,
        format=msg_format,
        metadataHeaders=metadata_headers
    ).execute()
    return message_detail

def get_file_data(message_id,attachment_id,file_name,save_location):
    response=service.users().messages().attachments().get(
        userId='me',
        messageId=message_id,
        id=attachment_id
    ).execute()

    file_data=base64.urlsafe_b64decode(response.get('data').encode('UTF-8'))
    return file_data
    
#from google.auth.transport.requests import Request
#creds.refresh(Request())


# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

creds = Credentials.from_authorized_user_file('../token.json', SCOPES)
service = build('gmail', 'v1', credentials=creds)


## search for emails with 'report' in the subject line

#query_string='has:attachment' # all email with attachments
query_string='filename:report*csv' #find all the email with attachment in csv format containing 'report' in their name
#query_string='report' # all email with report in subject/body text
email_messages = search_email(service, query_string,['INBOX'])
#print(email_messages)
try:
    path = os.path.join('./', 'Downloaded_files')
    os.mkdir(path)
except OSError as error:
    print(error) 
save_location='./Downloaded_files'
## download attachments for found emails 

for email_message in email_messages:
    #print(email_message['id'])
    #187ffb2fbf3c1bc5
    messageDetail = get_message_detail(email_message['id'],msg_format='full',metadata_headers=['parts'])
    #print(messageDetail)
    messageDetailPayload = messageDetail.get('payload')

    if 'parts' in messageDetailPayload:
        for msgPayload in messageDetailPayload['parts']:
            file_name = msgPayload['filename']
            body=msgPayload['body']

            if 'attachmentId' in body:
                attachment_id = body['attachmentId']
                attachment_content= get_file_data(email_message['id'],attachment_id,file_name,save_location)
                print(file_name)
                
                #with open(os.path.join(save_location + '/' 'Downloaded_files' + '/' + file_name),'wb') as _f:
                with open(os.path.join(save_location + '/' + file_name),'wb') as _f:
                    _f.write(attachment_content)
                    print(f'File {file_name} is saved at {save_location}')
    time.sleep(0.5)
    #break

[Errno 17] File exists: './Downloaded_files'
2022-09-15-23-04-08-EDT-Historical-Report-ENTRFacebook-2022-06-16--2022-09-16.csv
File 2022-09-15-23-04-08-EDT-Historical-Report-ENTRFacebook-2022-06-16--2022-09-16.csv is saved at ./Downloaded_files
2022-09-16-23-04-01-EDT-Historical-Report-ENTRFacebook-2022-06-17--2022-09-17.csv
File 2022-09-16-23-04-01-EDT-Historical-Report-ENTRFacebook-2022-06-17--2022-09-17.csv is saved at ./Downloaded_files
2022-09-17-23-04-42-EDT-Historical-Report-ENTRFacebook-2022-06-18--2022-09-18.csv
File 2022-09-17-23-04-42-EDT-Historical-Report-ENTRFacebook-2022-06-18--2022-09-18.csv is saved at ./Downloaded_files
2022-09-18-23-03-32-EDT-Historical-Report-ENTRFacebook-2022-06-19--2022-09-19.csv
File 2022-09-18-23-03-32-EDT-Historical-Report-ENTRFacebook-2022-06-19--2022-09-19.csv is saved at ./Downloaded_files
2022-09-20-23-03-35-EDT-Historical-Report-ENTRFacebook-2022-06-21--2022-09-21.csv
File 2022-09-20-23-03-35-EDT-Historical-Report-ENTRFacebook-2022-06-21--2

In [ ]:
import pandas as pd
import psycopg2
import os
import glob



In [ ]:
path

In [ ]:
#Write a python script that identifies the correct daily reports

!ls /Users/anya/SCIENCE/DW_Challenge/Junior_DE_Task/daily_reports/

#daily_repo_filepath= "/Users/anya/SCIENCE/DW_Challenge/Junior_DE_Task/daily_reports/"
daily_repo_filepath='./Downloaded_files' # downloaded files via API
#file1='2022-09-15-23-04-08-EDT-Historical-Report-ENTRFacebook-2022-06-16--2022-09-16.csv'
files=glob.glob(os.path.join(daily_repo_filepath,"*.csv"))
##date


In [ ]:
files

In [ ]:
data=pd.read_csv(daily_repo_filepath+file1)

In [ ]:
data

In [ ]:
# connect to postgres database
param_dic = {
    "host"      : "localhost",
    "database"  : "postgres",
    "user"      : "postgres",
}
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

In [ ]:
conn = connect(param_dic)

def execute_query(conn, query):
    """ Execute a single query """
    connect(param_dic)
    ret = 0 # Return value
    cursor = conn.cursor()
    try:
        cursor.execute(query)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1

    # If this was a select query, return the result
    if 'select' in query.lower():
        ret = cursor.fetchall()
    cursor.close()
    return ret

In [ ]:
#create postgres database

cols=','.join([column.replace(" ", "_").replace("?","").lower() + str(' VARCHAR') for column in data.columns.values])
new_table='new_table'

#query=("DROP TABLE IF EXISTS %s;"%(new_table))
query=("CREATE TABLE IF NOT EXISTS %s(%s)" %(new_table,(cols))) 

execute_query(conn, query)

In [ ]:
# 2nd option create postgres database
def create_postgres (db,new_table):
    cols=cols=','.join([column.replace(" ", "_").replace("?","").lower() + str(' VARCHAR') for column in data.columns.values])

    conn = connect(param_dic)
    cur = conn.cursor()

    #drop table if already exists
    cur.execute("DROP TABLE IF EXISTS %s;"%(new_table))
    #create database
    cur.execute("CREATE TABLE IF NOT EXISTS %s(%s)" %(new_table,(cols))) 
    conn.commit()
    conn.close()

In [ ]:
## data columns to create SQL database 
#VARCHAR for csv input
cols=','.join([column.replace(" ", "_").replace("?","").lower() + str(' VARCHAR') for column in data.columns.values])
cols
# for column in data.columns.values:
#     print(column.replace(" ", "_") + str(type(data[column][0])).replace("class", "").replace("<", "").replace(">", "").replace("'", "").replace("numpy.int64","int").replace("numpy.float64","float").replace("str","text") +',')    

#same with list comprehension
# types=([ column.replace(" ", "_") + str(type(data[column][0])).replace("class", "").replace("<", "").replace(">", "").replace("'", "").replace("numpy.int64","int").replace("numpy.float64","float").replace("str","text") for column in data.columns.values])

In [ ]:
## old create database
#connect database
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres")
cur = conn.cursor()

#create database
cur.execute("""
    CREATE TABLE IF NOT EXISTS reports_varchar(
    page_name VARCHAR,
    user_name VARCHAR,
    facebook_id VARCHAR,
    page_category VARCHAR,
    page_admin_top_country VARCHAR,
    page_description VARCHAR,
    page_created VARCHAR,
    likes_at_posting VARCHAR,
    followers_at_posting VARCHAR,
    post_created VARCHAR,
    post_created_date VARCHAR,
    post_created_time VARCHAR,
    type VARCHAR,
    total_interactions VARCHAR,
    likes VARCHAR,
    comments VARCHAR,
    shares VARCHAR,
    love VARCHAR,
    wow VARCHAR,
    haha VARCHAR,
    sad VARCHAR,
    angry VARCHAR,
    care VARCHAR,
    video_share_status VARCHAR,
    is_video_owner VARCHAR,
    post_views VARCHAR,
    total_views VARCHAR,
    total_views_for_all_crossposts VARCHAR,
    video_length VARCHAR,
    url VARCHAR,
    message VARCHAR,
    link VARCHAR,
    final_link VARCHAR,
    image_VARCHAR VARCHAR,
    link_VARCHAR VARCHAR,
    description VARCHAR,
    sponsor_id VARCHAR,
    sponsor_name VARCHAR,
    sponsor_category VARCHAR,
    overperforming_score_ VARCHAR
)
""")
conn.commit()
conn.close()

In [ ]:
## Upload data to the postgres database

def upload_csv_to_postgres(table, csv):
    conn = connect(param_dic)
    cur = conn.cursor()
    try:
        [cur.copy_expert("COPY %s FROM STDIN WITH DELIMITER ',' CSV "%(table), open(csv_file)) for csv_file in files]
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cur.close()
        return 1

    conn.commit()
    conn.close()

In [ ]:
table='new_table'

create_postgres(data,table)
upload_csv_to_postgres(table,files)


In [ ]:
conn = connect(param_dic)
execute_query(conn, 'SELECT Page_Name User_Name FROM new_table')

In [ ]:
##SQL script to process the data and push the data to GIT
## create tables 


# Task 2

#### Have a look at the dwh_dl_facebook_post_insights.csv. You will find there 3 rows for seperate posts. Import the csv to a database table

In [ ]:
path_insights='/Users/anya/SCIENCE/DW_Challenge/Junior_DE_Task/'
db_insights=pd.read_csv(path_insights+'dwh_dl_facebook_post_insights.csv')

In [ ]:
db_insights

In [ ]:
files

In [ ]:

table='insights_draft'
data=db_insights
files=glob.glob(os.path.join(path_insights,"*.csv"))
#conn = connect(param_dic)
create_postgres(data,table)
upload_csv_to_postgres(table,files)

In [ ]:
files

### Please write a SQL script to unpack the json in column post_video_view_time_by_age_bucket_and_gender in a new table, while preserving the other columns.

In [ ]:
SELECT 
   id, 
   data::json->'name' as name,
   data::json->'author' ->> 'last_name' as author
FROM books;

In [ ]:
conn = connect(param_dic)

sel_column='post_video_view_time_by_age_bucket_and_gender'
table='insights_draft'
execute_query(conn,'SELECT %s FROM %s'%(sel_column,table))

In [ ]:
execute_query(conn,'SELECT %s FROM %s'%(sel_column,table))

In [ ]:
execute_query(conn, "SELECT %s  FROM %s, data::jsons->'U.18-24' as U.18-24, data::jsons->'U.25-34' as U.25-34"%(sel_column,table))

In [ ]:
execute_query(conn,'SELECT %s FROM %s, json_each(%s)'%(sel_column,table,sel_column))

In [ ]:
print(sel_column)
conn = connect(param_dic)
execute_query(conn, "SELECT %s json_data -> '%s' %s, FROM %s;" %(sel_column,sel_column,'foo',table))

In [ ]:
table

In [ ]:
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres")
cur = conn.cursor()
cur.execute("""
    SELECT json_data 
        post_video_view_time_by_age_bucket_and_gender -> "U.18-24" as U1
    FROM insights_draft;
    """)
conn.commit()
conn.close()

In [ ]:
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres")
cur = conn.cursor()
cur.execute("""
    SELECT (post_video_view_time_by_age_bucket_and_gender) from insights_draft;
    """)
conn.commit()
conn.close()

In [ ]:
execute_query(conn,'select jsonb_object_keys(%s) from %s;'%(sel_column,table))

In [ ]:
table

In [ ]:
print(sel_column)
conn = connect(param_dic)
execute_query(conn, "SELECT %s -> 'U.18-24' AS U1 FROM %s" %(sel_column,table))

#SELECT post_video_view_time_by_age_bucket_and_gender FROM insights_draft;

#SELECT post_video_view_time_by_age_bucket_and_gender json_data ->> 'U.18-24' AS U1 FROM insights_draft; 

In [ ]:
execute_query(conn,"select 
    json_data #> 
       '{pizzas,1,additionalToppings}'  
        as additional_toppings_2nd_pizza
from test;")

### Write a script for the sums per post for each gender. Additionally create the sum for age 18-34 of all genders.

In [ ]:
# SELECT SUM(Quantity) FROM OrderDetails;
# add where statement WHERE Country='Mexico' WHERE Country='Mexico'
# SELECT * FROM Products WHERE Price BETWEEN 50 AND 60;